In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import cv2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Flatten
from tensorflow.keras.layers import Convolution2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

2022-06-29 16:27:19.538740: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


In [2]:
train="../input/fer2013/train"
test="../input/fer2013/test"
train_dgen=ImageDataGenerator(rescale=1./255)
test_dgen=ImageDataGenerator(rescale=1./255)

In [3]:
train_gen=train_dgen.flow_from_directory(
    train,
    target_size=(48,48),
    batch_size=64,
    color_mode="grayscale",
    class_mode="categorical"
)
test_gen=test_dgen.flow_from_directory(
    test,
    target_size=(48,48),
    batch_size=64,
    color_mode="grayscale",
    class_mode="categorical"
)

Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


In [4]:
model=Sequential()
model.add(Convolution2D(32,kernel_size=(3,3),activation='relu',input_shape=(48,48,1)))
model.add(Convolution2D(64,kernel_size=(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Convolution2D(128,kernel_size=(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Convolution2D(128,kernel_size=(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(1024,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(7,activation="softmax"))

2022-06-29 16:27:52.437734: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-06-29 16:27:52.441436: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-06-29 16:27:52.508200: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-29 16:27:52.509337: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla P100-PCIE-16GB computeCapability: 6.0
coreClock: 1.3285GHz coreCount: 56 deviceMemorySize: 15.90GiB deviceMemoryBandwidth: 681.88GiB/s
2022-06-29 16:27:52.509384: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2022-06-29 16:27:52.564340: I tensorflow/stream_executor/platform/def

In [5]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [6]:
train_model=model.fit_generator(
    train_gen,
    steps_per_epoch=28709 // 64,
    epochs=50,
    validation_data=test_gen,
    validation_steps=7178 // 64
)

/opt/conda/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '
2022-06-29 16:27:55.617821: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-06-29 16:27:55.622364: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2199995000 Hz


Epoch 1/50


2022-06-29 16:27:56.416876: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.11
2022-06-29 16:27:57.634206: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublasLt.so.11
2022-06-29 16:27:57.654233: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.8


448/448 [==============================] - 183s 390ms/step - loss: 1.7937 - accuracy: 0.2674 - val_loss: 1.5423 - val_accuracy: 0.3922
Epoch 2/50
448/448 [==============================] - 33s 74ms/step - loss: 1.5293 - accuracy: 0.4063 - val_loss: 1.3099 - val_accuracy: 0.4997
Epoch 3/50
448/448 [==============================] - 33s 73ms/step - loss: 1.3367 - accuracy: 0.4877 - val_loss: 1.2477 - val_accuracy: 0.5253
Epoch 4/50
448/448 [==============================] - 33s 74ms/step - loss: 1.2504 - accuracy: 0.5265 - val_loss: 1.1804 - val_accuracy: 0.5488
Epoch 5/50
448/448 [==============================] - 33s 73ms/step - loss: 1.1788 - accuracy: 0.5545 - val_loss: 1.1460 - val_accuracy: 0.5672
Epoch 6/50
448/448 [==============================] - 33s 73ms/step - loss: 1.1315 - accuracy: 0.5724 - val_loss: 1.1199 - val_accuracy: 0.5744
Epoch 7/50
448/448 [==============================] - 34s 75ms/step - loss: 1.1004 - accuracy: 0.5845 - val_loss: 1.1076 - val_accuracy: 0.5759
E

In [7]:
model.save('model.h5') 